In [43]:
#Dependencies
import numpy as np
import pandas as pd
import datetime as dt
import re

In [2]:
os.getcwd()

'/Users/jacosta3/OneDrive - University of South Florida/Gerdau Projects/pft/scripts'

## Fetching Indicators info

In [89]:
#File directory
ser_path = "./serialized/"
path = "./confidential/"
#change directory to datasets
os.chdir("../datasets")

In [90]:
indicators = pd.read_excel(path+"market_indicators.xlsx",sheet_name="Monthly")

In [91]:
indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 17 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   dd/mm/yy                                             124 non-null    datetime64[ns]
 1   Year                                                 124 non-null    int64         
 2   Month                                                124 non-null    object        
 3   National Architect Billing/monthly                   121 non-null    float64       
 4   Total Federal Construction/Billion $                 121 non-null    float64       
 5   Total State and Local Construction/Billion $         121 non-null    float64       
 6   US Steel Capacity Utilization                        109 non-null    float64       
 7   US Long Steel/million $ per short ton                26 non-null     float64       
 8   

## Cleaning Columns

In [92]:
def clean_columns(dataframe):
    for col in dataframe.columns:
        dataframe.rename(columns={col:re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', col)}, inplace=True)
    dataframe.columns = dataframe.columns.str.strip().str.lower().str.replace(" ","_")
    try:
        dataframe["calendar_day"] = dataframe["calendar_day"].astype("datetime64")
    except:
        pass
    try:
        dataframe["requested_date"] = dataframe["requested_date"].astype("datetime64")
        dataframe["confirmed_date"] = dataframe["confirmed_date"].astype("datetime64")
        dataframe["material_avail_date"] = dataframe["material_avail_date"].astype("datetime64")
        dataframe["load_date"] = dataframe["load_date"].astype("datetime64")
        dataframe["plan_goods_issue_date"] = dataframe["plan_goods_issue_date"].astype("datetime64")
    except:
        pass
    try:
        dataframe["snapshot_date"] = dataframe["snapshot_date"].astype("datetime64")
    finally:
        return dataframe.columns

In [93]:
clean_columns(indicators)

Index(['dd/mm/yy', 'year', 'month', 'national_architect_billing/monthly',
       'total_federal_construction/billion_$',
       'total_state_and_local_construction/billion_$',
       'us_steel_capacity_utilization',
       'us_long_steel/million_$_per_short_ton',
       'us_domestic_hot-rolled_coil/million_$_per_short_ton',
       'msci_carbon_bar_monthly_shipments',
       'msci_carbon_bar_monthly_shipments_(canada)',
       'structural_shipments_/usa', 'structural_shipments_/canada',
       'ism_manufacturing', 'ism_non_manufacturing',
       'global_steel_utilization', 'chicago#1_scrap_bushelling'],
      dtype='object')

### Create Month & Year Columns

In [94]:
indicators['year'] = indicators['dd/mm/yy'].dt.year
indicators['month'] = indicators['dd/mm/yy'].dt.month

### Rename Columns

In [96]:
indicators.columns

Index(['dd/mm/yy', 'year', 'month', 'national_architect_billing/monthly',
       'total_federal_construction/billion_$',
       'total_state_and_local_construction/billion_$',
       'us_steel_capacity_utilization',
       'us_long_steel/million_$_per_short_ton',
       'us_domestic_hot-rolled_coil/million_$_per_short_ton',
       'msci_carbon_bar_monthly_shipments',
       'msci_carbon_bar_monthly_shipments_(canada)',
       'structural_shipments_/usa', 'structural_shipments_/canada',
       'ism_manufacturing', 'ism_non_manufacturing',
       'global_steel_utilization', 'chicago#1_scrap_bushelling'],
      dtype='object')

In [98]:
#Remane column & pick only desired ones 
indicators = indicators.rename(columns={'dd/mm/yy':'calendar_day',
                                        'total_federal_construction/billion_$':'fed_construction$B',                                                              'total_state_and_local_construction/billion_$':'state_local_construction$B',
                                        'msci_carbon_bar_monthly_shipments':'carbon_bar_ship_usa',
                                        'msci_carbon_bar_monthly_shipments_(canada)':'carbon_bar_ship_canada',
                                        'structural_shipments_/usa':'structural_ship_usa', 
                                        'structural_shipments_/canada':'structural_ship_canada',
                                        'chicago#1_scrap_bushelling':'chicago_scrap$ST'})
indicators = indicators[['calendar_day','year','month', 'global_steel_utilization','fed_construction$B',                                           'state_local_construction$B','carbon_bar_ship_usa','carbon_bar_ship_canada',                                              'structural_ship_usa','structural_ship_canada', 'ism_manufacturing', 'ism_non_manufacturing',                             'chicago_scrap$ST']]

### Selecting 2016 dates on

In [99]:
#select 2016-2020
indicators = indicators[(indicators['calendar_day'].dt.year >= 2016) & (indicators['calendar_day'] <='2020-01-01')]
indicators = indicators.reset_index()

In [100]:
indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   index                       49 non-null     int64         
 1   calendar_day                49 non-null     datetime64[ns]
 2   year                        49 non-null     int64         
 3   month                       49 non-null     int64         
 4   global_steel_utilization    45 non-null     float64       
 5   fed_construction$B          49 non-null     float64       
 6   state_local_construction$B  49 non-null     float64       
 7   carbon_bar_ship_usa         49 non-null     float64       
 8   carbon_bar_ship_canada      49 non-null     float64       
 9   structural_ship_usa         49 non-null     float64       
 10  structural_ship_canada      49 non-null     float64       
 11  ism_manufacturing           49 non-null     float64       
 

### Fill NaNs

In [101]:
#filling NaNs - assumption gradual increase 
values = {36:0.7768,37:0.7915,38:0.8063,39:0.8210}
indicators['global_steel_utilization'] = indicators['global_steel_utilization'].fillna(value=values, axis=0)

In [102]:
indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   index                       49 non-null     int64         
 1   calendar_day                49 non-null     datetime64[ns]
 2   year                        49 non-null     int64         
 3   month                       49 non-null     int64         
 4   global_steel_utilization    49 non-null     float64       
 5   fed_construction$B          49 non-null     float64       
 6   state_local_construction$B  49 non-null     float64       
 7   carbon_bar_ship_usa         49 non-null     float64       
 8   carbon_bar_ship_canada      49 non-null     float64       
 9   structural_ship_usa         49 non-null     float64       
 10  structural_ship_canada      49 non-null     float64       
 11  ism_manufacturing           49 non-null     float64       
 

In [105]:
indicators.head()

,index,calendar_day,year,month,global_steel_utilization,fed_construction$B,state_local_construction$B,carbon_bar_ship_usa,carbon_bar_ship_canada,structural_ship_usa,structural_ship_canada,ism_manufacturing,ism_non_manufacturing,chicago_scrap$ST
0,72,2016-01-01,2016,1,0.660,1583.0,16684.0,204300.0,30800.0,175300.0,38700.0,48.2,53.5,180.0
1,73,2016-02-01,2016,2,0.660,1840.0,17198.0,205400.0,31000.0,187000.0,37500.0,49.5,53.4,180.0
2,74,2016-03-01,2016,3,0.707,1844.0,18555.0,221200.0,31400.0,203900.0,38400.0,51.8,54.5,195.0
3,75,2016-04-01,2016,4,0.705,1703.0,20187.0,204600.0,31200.0,193800.0,37400.0,50.8,55.7,245.0
4,76,2016-05-01,2016,5,0.713,1888.0,21954.0,200500.0,29700.0,191440.0,36600.0,51.3,52.9,275.0


## Save to file

In [104]:
#indicators.to_pickle(ser_path+'indicators.pkl')

## Ready for merging 
Consider whether monthly value ingestion may be better than weekly

### Importing Structural df

In [106]:
structural = pd.read_pickle(ser_path+"structural_merged.pkl")

In [107]:
structural.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158270 entries, 0 to 158269
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   calendar_day              158270 non-null  datetime64[ns]
 1   year                      158270 non-null  object        
 2   month                     158270 non-null  object        
 3   week                      158270 non-null  object        
 4   block                     158270 non-null  object        
 5   size                      158270 non-null  object        
 6   IN_tons                   158270 non-null  float64       
 7   SO_order_qty_sales_units  158270 non-null  float64       
 8   SH_shipment_tons          158270 non-null  float64       
 9   PR_ton                    158270 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(5)
memory usage: 12.1+ MB


In [110]:
#dropping columns to permit merging
structural = structural.drop(columns=['year','month','week'])

In [124]:
structural.columns

Index(['calendar_day', 'block', 'size', 'IN_tons', 'SO_order_qty_sales_units',
       'SH_shipment_tons', 'PR_ton'],
      dtype='object')

### Creating Weekly dataframe for merging

In [118]:
weekly = structural.groupby('block').resample('W', on='calendar_day').sum().sort_values('calendar_day').reset_index()

In [125]:
weekly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12519 entries, 0 to 12518
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   calendar_day              12519 non-null  datetime64[ns]
 1   block                     12519 non-null  object        
 2   IN_tons                   12519 non-null  float64       
 3   PR_ton                    12519 non-null  float64       
 4   SO_order_qty_sales_units  12519 non-null  float64       
 5   SH_shipment_tons          12519 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 587.0+ KB


In [129]:
weekly.head(10)

,calendar_day,year,month,block,IN_tons,PR_ton,SO_order_qty_sales_units,SH_shipment_tons
0,2016-01-03,2016,1,"3-1/2"" ANG",207.261,0.0,0.000,0.0
1,2016-01-03,2016,1,"9"" CHN",43.296,0.0,0.000,0.0
2,2016-01-03,2016,1,"8 X 4"" ANG",14.112,0.0,0.000,0.0
3,2016-01-03,2016,1,"6"" CHN",1857.997,0.0,0.000,0.0
4,2016-01-03,2016,1,"4 X 3"" ANG",472.990,0.0,0.000,0.0
5,2016-01-03,2016,1,"5 X 3-1/2"" ANG",1888.340,0.0,0.000,0.0
6,2016-01-03,2016,1,"6"" FLT",1102.054,0.0,2.244,0.0
7,2016-01-03,2016,1,"8"" FLT",867.865,0.0,2.244,0.0
8,2016-01-03,2016,1,"5 X 3"" ANG",1172.033,0.0,0.000,0.0
9,2016-01-03,2016,1,"8"" CHN",958.505,0.0,0.000,0.0


In [126]:
#recreating time columns for merging
weekly['month'] = weekly['calendar_day'].dt.month
weekly['year'] = weekly['calendar_day'].dt.year

In [127]:
#weekly data ready for merging
weekly = weekly[['calendar_day','year','month','block','IN_tons','PR_ton','SO_order_qty_sales_units','SH_shipment_tons']]
weekly.head()

,calendar_day,year,month,block,IN_tons,PR_ton,SO_order_qty_sales_units,SH_shipment_tons
0,2016-01-03,2016,1,"3-1/2"" ANG",207.261,0.0,0.0,0.0
1,2016-01-03,2016,1,"9"" CHN",43.296,0.0,0.0,0.0
2,2016-01-03,2016,1,"8 X 4"" ANG",14.112,0.0,0.0,0.0
3,2016-01-03,2016,1,"6"" CHN",1857.997,0.0,0.0,0.0
4,2016-01-03,2016,1,"4 X 3"" ANG",472.990,0.0,0.0,0.0
